# Исследование надежности заемщиков

Импорт pandas и чтение файла 

In [2]:
import pandas as pd

data = pd.read_csv('/Users/alinadavydenko/Desktop/sql_hw/analysis_borrower_reliability/dataset/data.csv')

print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


# Предобработка данных

Удаление пропусков 
Количество пропущенных значений для каждого столбца

In [4]:
count_na = data.isnull().sum()  
print(count_na)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


На сумму дохода сильнее всего влияет тип занятости, поэтому заполню пропуски в столбце days_employed медианным значением по каждому типу из столбца income_type

In [5]:
# Находим медианы
median_incomes = data.groupby('income_type')['total_income'].median()

# Создаем Series с медианами для каждой строки
medians_series = data['income_type'].map(median_incomes)

# Заполняем пропуски
data['total_income'] = data['total_income'].fillna(medians_series)

print(medians_series)

0        142594.396847
1        142594.396847
2        142594.396847
3        142594.396847
4        118514.486412
             ...      
21520    172357.950966
21521    118514.486412
21522    142594.396847
21523    142594.396847
21524    142594.396847
Name: income_type, Length: 21525, dtype: float64


# Обработка аномальных значений 

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведу медианное значение трудового стажа days_employed в днях

In [7]:
median_days_employed = data.groupby('income_type')['days_employed'].median()
print(median_days_employed)

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


Уникальные значения children

In [8]:
unique_children = data['children'].unique()

print(unique_children)

[ 1  0  3  2 -1  4 20  5]


В столбце children есть два аномальных значения. Удалю строки, в которых встречаются такие аномальные значения из датафрейма data

In [9]:
data = data.loc[(data['children'] >= 0) & (data['children'] < 10)]

print(data['children'].unique())

[1 0 3 2 4 5]


Заполню пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [10]:
# находиммедианное среднее 
type_median = data.groupby('income_type')['days_employed'].median()

# Заполняем пропуски
data['days_employed'] = data['days_employed'].fillna(data['income_type'].map(type_median))

print(data['days_employed'].isnull().sum())

0


# Изменение типов данных

Замена вещественного тип данных в столбце total_income на целочисленный с помощью метода astype()

In [11]:
data['total_income'] = data['total_income'].astype('int')

# Обработка дубликатов

Обработка неявных дубликатов в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

In [12]:
data['education'] = data['education'].str.lower()

dubl_data = data.duplicated().sum()
print(dubl_data)

71


In [13]:
data = data.drop_duplicates()

# Категоризация данных

Датафрейм data столбец total_income_category с категориями:

0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

In [14]:
def categorize_income(row):
    if 0 <= row <= 30000:
        return 'E'
    elif 30001 <= row <= 50000:
        return 'D'
    elif 50001 <= row <= 200000:
        return 'C'
    elif 200001 <= row <= 1000000:
        return 'B'
    else:
        return 'A'
    
data['total_income_category'] = data['total_income'].apply(categorize_income)

print(data['total_income_category'].value_counts())

total_income_category
C    15921
B     5014
D      349
A       25
E       22
Name: count, dtype: int64


Функция, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.


In [15]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [17]:
def categorize_purpose(purpose):
    purpose_lower = purpose.lower()
    if 'автомобил' in purpose_lower:
        return 'операции с автомобилем'
    elif 'жил' in purpose_lower or 'недвижим' in purpose_lower or 'строительств' in purpose_lower:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose_lower:
        return 'проведение свадьбы'
    elif 'образован' in purpose_lower:
        return 'получение образования'
    else:
        return 'другое'
    
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

print(data['purpose_category'].value_counts())

purpose_category
операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: count, dtype: int64
